# Practice Exercises

## Import Libraries

In [28]:
import random

from langgraph.graph import START, END, StateGraph, MessagesState
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import ToolNode
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage, ToolMessage
from langchain_core.tools import tool
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
from IPython.display import Image, display
from typing import Literal
import os   

from ddgs import DDGS

print("All imports successful")

All imports successful


In [2]:
load_dotenv()
api_key = os.getenv('paid_api')

if not api_key:
    raise ValueError("API key not found!")
print("API Key loaded successfully")

API Key loaded successfully


In [3]:
llm = ChatOpenAI(
    model = "gpt-4o-mini",
    temperature = 0.7,
    api_key=api_key
)

print(f"LLM initialized {llm.model_name}")

LLM initialized gpt-4o-mini


## Creating Custom Tools

In [4]:

@ tool
def weather_tool(city: str) -> str:
    """
    Returns simulated weather for a given city.
    Use this tool when the weather of a given city is required.

    Args:
        city: The given city
    
    Returns:
        A simulated weather for the city
    """

    def simulate_weather(city: str) ->dict:
        """
        Generate deterministic fake weather data for a city.
        """
        random.seed(city.lower())

        conditions = ["Sunny", "Cloudy", "Rainy", "Thunderstorm", "Hazy"]
        
        return {
            "city": city.title(),
            "temperature_c": random.randint(22, 36),
            "condition": random.choice(conditions),
            "humidity_percent": random.randint(40, 90)
        }
    
    weather = simulate_weather(city)
    return (
        f"Weather in {weather['city']}:\n"
        f"- Temperature: {weather['temperature_c']}°C\n"
        f"- Condition: {weather['condition']}\n"
        f"- Humidity: {weather['humidity_percent']}%"
    )
print("Weather tool defined.")

Weather tool defined.


In [5]:
@tool
def dictionary(question: str) -> str:
    """
    Accepts a question and returns a definition from a simulated dictionary.

    Example inputs:
    - "What is the meaning of agent?"
    - "Define RAG"
    - "What does embedding mean?"
    """

    # Simulated dictionary
    dictionary = {
        "agent": "An entity that perceives its environment and acts upon it.",
        "tool": "A function or capability an agent can use to perform a task.",
        "rag": "Retrieval-Augmented Generation, combining retrieval with generation.",
        "llm": "Large Language Model trained on vast amounts of text data.",
        "embedding": "A numerical representation of text capturing semantic meaning.",
        "help": "To give assistance or support to (someone)"
    }

    # Normalize question
    word = question.lower().strip()


    # Lookup
    if word in dictionary:
        return f"{word.title()}: {dictionary[word]}"
    else:
        return f"{word.title()}: Definition not found in the simulated dictionary."
print("Dictionary tool defined")

Dictionary tool defined


In [ ]:

@tool
def web_search(query: str) -> str:
    """
    Searches the web using DuckDuckGo and returns summarized results.

    Example inputs:
    - "Search the internet for the latest news on AI"
    - "What is LangGraph?"
    - "Weather patterns in Nigeria"
    """


    ddgs = DDGS()
    results = ddgs.text(query, max_results =4)

    if not results:
        return f"No results found for {query}"
    output = f'Search Results for "{query}":\n\n'

    for result in results:
        output += result['title']
        output += result['body']
        output += result['href']
    return output

print("Web search tool defined")


Web search tool defined


## Binding tools to LLM

In [30]:
tools= [weather_tool, dictionary, web_search]

llm_with_tools = llm.bind_tools(tools)
print(f"LLM bound to {len(tools)} tools")
print(f"Tools: {[tool.name for tool in tools]}")

LLM bound to 3 tools
Tools: ['weather_tool', 'dictionary', 'web_search']


## Defining the Assistant Node

In [31]:
sys_msg = SystemMessage(content="""You are a helpful assitant with access to tools.
When asked to perform for the weather, use the weather tool.
When asked to define something, use the dictionary tool.
When asked to search the internet, use the web search tool.

Only use tools when necessary - for simple questions, answer directly.""")

def assistant(state: MessagesState) -> dict:
    """
    Assistant node - decides whether to use tools or answer directly.
    """
    messages = [sys_msg] + state["messages"]
    response = llm_with_tools.invoke(messages)
    return {"messages": [response]}

print("Assistant mode defined")

Assistant mode defined


## Implementing Conditional Routing

In [32]:
def should_continue(state: MessagesState)-> Literal["tools", "__end__"]:
    """
    Decide next step based on last message.
    
    If LLM called a tool → go to 'tools' node
    If LLM provided final answer → go to END
    """
    last_message = state["messages"][-1]

    # check if LLM made tool calls
    if last_message.tool_calls:
        return "tools"
    return "__end__"
print("Conditional routing function defined")

Conditional routing function defined


In [33]:
builder = StateGraph(MessagesState)

builder.add_node("assistant", assistant)
builder.add_node("tools", ToolNode(tools))

builder.add_edge(START, "assistant")
builder.add_conditional_edges(
    "assistant",
    should_continue,
    {"tools": "tools", "__end__": END}
)
builder.add_edge("tools", "assistant")

memory = MemorySaver()
agent= builder.compile(checkpointer=memory)

print("Agent graph compiled with tools and memory")

Agent graph compiled with tools and memory


## The Agent

In [34]:
def my_agent(user_input: str, thread_id: str = "test7"):
    """
    Run the agent and display the conversation.
    """
    print(f"\n{'='*70}")
    print(f"User: {user_input}")
    print(f"{'='*70}\n")

    result = agent.invoke(
        {"messages": [HumanMessage(content=user_input)]},
        config={"configurable": {"thread_id": thread_id}}
    )

    for message in result["messages"]:
        if isinstance(message, HumanMessage):
            continue
        elif isinstance(message, AIMessage):
            if message.tool_calls:
                print(f"\nAgent: [Calling tool: {message.tool_calls[0]["name"]}]")
            else:
                print(f"\nAgent: {message.content}")
        elif isinstance(message, ToolMessage):
            print(f"\nTool Result: {message.content[:50]}...")
    print(f"\n{'='*70}\n")

In [36]:
my_agent("What is the weather in Ogun?", thread_id="demo0")


User: What is the weather in Ogun?


Agent: [Calling tool: weather_tool]

Tool Result: Weather in Ogun:
- Temperature: 27°C
- Condition: ...

Agent: The weather in Ogun is currently 27°C, with cloudy conditions and a humidity level of 52%.




In [35]:
my_agent("Search for news about Venezuela", thread_id="demo2")


User: Search for news about Venezuela


Agent: [Calling tool: web_search]

Tool Result: Search Results for "news about Venezuela":

Google...

Agent: Here are some recent news highlights about Venezuela:

1. **Purchase of American Products**: Venezuela is moving towards purchasing only American-made products, using funds from a new oil deal. [Read more](https://news.google.com/stories/CAAqNggKIjBDQklTSGpvSmMzUnZjbmt0TXpZd1NoRUtEd2lEMWF1bkVCRUxoOXdxbGNGOEZTZ0FQAQ?hl=en-NG&gl=NG&ceid=NG:en).

2. **BBC Coverage**: The BBC has extensive coverage and analysis on various topics related to Venezuela. [Explore here](https://www.bbc.com/news/topics/cg41ylwvwgxt).

3. **Venezuelanalysis**: This platform provides independent news and analysis on Venezuela, supported by readers. [Check it out](https://venezuelanalysis.com/).

4. **Airline Permits Revoked**: The Venezuelan government has revoked operating permits for six airlines, including Iberia, TAP, Avianca, and Latam Colombia, due to the canc